In [1]:
# Load Model (Training)
import torch
from cldm.toss import TOSS
from cldm.model import load_state_dict
from omegaconf import OmegaConf

from ldm.util import instantiate_from_config

config = OmegaConf.load("models/toss_vae.yaml")
model = instantiate_from_config(config.model)

state_dict = load_state_dict("ckpt/toss.ckpt")

# keys_to_remove = [
#     k for k in state_dict.keys()
#     if "pose_net" in k
# ]

# for k in keys_to_remove:
#     print("Removing:", k)
#     del state_dict[k]

m, u = model.load_state_dict(state_dict, strict=False)

print("Missing keys:", m)
print("Unexpected keys:", u)

c:\Users\user\.conda\envs\toss\lib\site-packages\lightning_fabric\__init__.py:29: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)
c:\Users\user\.conda\envs\toss\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No module 'xformers'. Proceeding without it.


c:\Users\user\.conda\envs\toss\lib\site-packages\pytorch_lightning\utilities\distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v2.0.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from C:\Users\user\_netrc.
wandb: Currently logged in as: wndyd0131 (wndyd0131-sungkyunkwan-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


TossLoraModule: Running in eps-prediction mode
DiffusionWrapper has 863.65 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
[INIT] Fixing zero-initialized lora_B: base_model.model.middle_block.1.transformer_blocks.0.attn2.to_q.lora_B.default.weight
[INIT] Fixing zero-initialized lora_B: base_model.model.middle_block.1.transformer_blocks.0.attn2.to_k.lora_B.default.weight
[INIT] Fixing zero-initialized lora_B: base_model.model.middle_block.1.transformer_blocks.0.attn2.to_v.lora_B.default.weight
[INIT] Fixing zero-initialized lora_B: base_model.model.output_blocks.3.1.transformer_blocks.0.attn2.to_q.lora_B.default.weight
[INIT] Fixing zero-initialized lora_B: base_model.model.output_blocks.3.1.transformer_blocks.0.attn2.to_k.lora_B.default.weight
[INIT] Fixing zero-initialized lora_B: base_model.model.output_blocks.3.1.transformer_blocks.0.attn2.to_v.lora_B.d

In [2]:
trainable = [n for n, p in model.named_parameters() if p.requires_grad]
print("Trainable parameters:", len(trainable))
for n in trainable:
    print(n)

Trainable parameters: 16
model.diffusion_model.base_model.model.middle_block.1.transformer_blocks.0.attn2.to_q.lora_A.default.weight
model.diffusion_model.base_model.model.middle_block.1.transformer_blocks.0.attn2.to_q.lora_B.default.weight
model.diffusion_model.base_model.model.middle_block.1.transformer_blocks.0.attn2.to_k.lora_A.default.weight
model.diffusion_model.base_model.model.middle_block.1.transformer_blocks.0.attn2.to_k.lora_B.default.weight
model.diffusion_model.base_model.model.middle_block.1.transformer_blocks.0.attn2.to_v.lora_A.default.weight
model.diffusion_model.base_model.model.middle_block.1.transformer_blocks.0.attn2.to_v.lora_B.default.weight
model.diffusion_model.base_model.model.output_blocks.3.1.transformer_blocks.0.attn2.to_q.lora_A.default.weight
model.diffusion_model.base_model.model.output_blocks.3.1.transformer_blocks.0.attn2.to_q.lora_B.default.weight
model.diffusion_model.base_model.model.output_blocks.3.1.transformer_blocks.0.attn2.to_k.lora_A.default.w

In [3]:
import numpy as np

def rotation_matrix_to_euler(R):
    """
    Extract pitch (x-rotation) and yaw (y-rotation) from a 3x3 rotation matrix.
    Returns angles in radians.
    
    Assumes rotation order: R = Ry(yaw) @ Rx(pitch) @ Rz(roll)
    """
    # Clamp to avoid numerical issues with asin
    sy = np.clip(R[0, 2], -1.0, 1.0)
    yaw = np.arcsin(sy)
    
    # Check for gimbal lock
    if np.abs(sy) < 0.99999:
        pitch = np.arctan2(-R[1, 2], R[2, 2])
    else:
        pitch = np.arctan2(R[2, 1], R[1, 1])
    
    return pitch, yaw


def pose_matrix_to_toss_format(pose_4x4):
    """
    Convert a 4x4 pose matrix to TOSS format: [pitch, yaw, distance]
    
    Args:
        pose_4x4: 4x4 transformation matrix (camera-to-world or world-to-camera)
    
    Returns:
        [pitch, yaw, distance] as expected by TOSS pose_enc="vae" or "freq"
    """
    R = pose_4x4[:3, :3]  # 3x3 rotation
    t = pose_4x4[:3, 3]   # translation vector
    
    pitch, yaw = rotation_matrix_to_euler(R)
    
    # Distance: typically the Z component or the norm of translation
    # Adjust based on your coordinate system
    distance = np.linalg.norm(t)  # or t[2] if Z is the depth axis
    
    return np.array([pitch, yaw, distance], dtype=np.float32)


def compute_relative_pose(src_pose_4x4, tgt_pose_4x4):
    """
    Compute relative pose from source to target view.
    Returns [delta_pitch, delta_yaw, delta_distance] in radians.
    """
    src_pitch, src_yaw = rotation_matrix_to_euler(src_pose_4x4[:3, :3])
    tgt_pitch, tgt_yaw = rotation_matrix_to_euler(tgt_pose_4x4[:3, :3])
    
    src_dist = np.linalg.norm(src_pose_4x4[:3, 3])
    tgt_dist = np.linalg.norm(tgt_pose_4x4[:3, 3])
    
    delta_pitch = tgt_pitch - src_pitch
    delta_yaw = tgt_yaw - src_yaw
    delta_distance = tgt_dist - src_dist
    
    return np.array([delta_pitch, delta_yaw, delta_distance], dtype=np.float32)

In [4]:
# Portrait4D Dataset Class

import random
import os
from pathlib import Path
from PIL import Image
import numpy as np
import torch
from torchvision import transforms as T

# root
    # 251514-296055
        # src_img.png
        # 00000.jpg-00020.jpg # tgt
        # 00000_mask.png-00020.png # mask
        # poses.npy # pose
        
        # src_img
        # tgt_img
        # mask
        # delta_pose
        # txt
import os

class Portrait4dDataset:
    def __init__(self, root_dir, transform=None):
        # self.root = Path(root)
        self.root_dir = root_dir
        self.transform = transform
        self.subjects = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.samples = []
        
        for sub in self.subjects:
            sub_path = os.path.join(root_dir, sub)
            views = sorted([f for f in os.listdir(sub_path) if f.endswith('.jpg') and not f.startswith('src')])

            for view_file in views:
                view_id = view_file.split('.')[0]
                mask_file = f"{view_id}_mask.png"
                
                if os.path.exists(os.path.join(sub_path, mask_file)):
                    self.samples.append({
                        'sub_path': sub_path,
                        'view_file': view_file,
                        'mask_file': mask_file,
                        'view_index': int(view_id)
                    })
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        sub_path = sample['sub_path']
        src_idx = 9
        
        # Load Source Image
        src_path = os.path.join(sub_path, "src.jpg")
        src = Image.open(src_path).convert("RGB")

        # Load View Image
        img_path = os.path.join(sub_path, sample['view_file'])
        image = Image.open(img_path).convert("RGB")
        
        # Load Mask
        mask_path = os.path.join(sub_path, sample['mask_file'])
        mask = Image.open(mask_path).convert("L") # 1-channel grayscale
        
        # Load Pose
        poses = np.load(os.path.join(sub_path, 'poses.npy'))

        if poses.ndim == 2 and poses.shape[1] == 16:
            poses = poses.reshape(-1, 4, 4)

        src_pose = poses[src_idx]
        
        # Target: The pose of the current view (000xx.jpg)
        tgt_pose = poses[sample['view_index']]
        
        # Delta: The movement required to get from 'hint' to 'target'
        delta_pose = compute_relative_pose(src_pose, tgt_pose)
        delta_pose = torch.from_numpy(delta_pose).float()
        
        if self.transform:
            src = self.transform(src)
            image = self.transform(image)
            mask_transform = T.Compose([
                T.Resize((256, 256)),
                T.ToTensor()
            ])
            mask = mask_transform(mask)
    
        return {
            "jpg": image,
            "hint": src,
            'mask': mask,
            "delta_pose": delta_pose, # pose from original angle
            "subject_id": os.path.basename(sub_path),
            "txt": ""
        }

In [5]:
from torch.utils.data import dataloader

# Debug: Verify pose extraction is working correctly
# Run this BEFORE training to validate your data

import numpy as np

from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader

transform = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    # T.Normalize([0.5], [0.5])
])

dataset = Portrait4dDataset(
    root_dir="datasets/portrait4d", transform=transform
)

dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)

# Load one subject's poses to verify
sample_subject = dataset.subjects[0]
sample_path = os.path.join(dataset.root_dir, sample_subject, 'poses.npy')
poses = np.load(sample_path)

print(f"Raw poses shape: {poses.shape}")

# Reshape if flattened (N, 16) -> (N, 4, 4)
if poses.ndim == 2 and poses.shape[1] == 16:
    poses = poses.reshape(-1, 4, 4)
    print(f"Reshaped to: {poses.shape}")

print(f"Number of views: {poses.shape[0]}")

# Print one pose matrix to verify it looks like a valid transformation matrix
print(f"\nSample pose matrix (view 0):\n{poses[0]}")

# Check a few poses
src_idx = 9
for tgt_idx in [0, 5, 10, 15]:
    if tgt_idx < poses.shape[0]:
        src_pose = poses[src_idx]
        tgt_pose = poses[tgt_idx]
        
        delta = compute_relative_pose(src_pose, tgt_pose)
        print(f"\nSource view {src_idx} -> Target view {tgt_idx}:")
        print(f"  delta_pitch: {np.degrees(delta[0]):.2f}°")
        print(f"  delta_yaw:   {np.degrees(delta[1]):.2f}°")
        print(f"  delta_dist:  {delta[2]:.4f}")

# Verify a sample from dataloader
sample_batch = next(iter(dataloader))
print(f"\n--- Sample batch ---")
print(f"jpg shape: {sample_batch['jpg'].shape}")
print(f"hint shape: {sample_batch['hint'].shape}")
print(f"delta_pose shape: {sample_batch['delta_pose'].shape}")
print(f"delta_pose sample values (radians): {sample_batch['delta_pose'][0]}")
print(f"delta_pose sample values (degrees): pitch={np.degrees(sample_batch['delta_pose'][0, 0].item()):.2f}°, yaw={np.degrees(sample_batch['delta_pose'][0, 1].item()):.2f}°")

Raw poses shape: (20, 16)
Reshaped to: (20, 4, 4)
Number of views: 20

Sample pose matrix (view 0):
[[ 9.5087630e-01  3.8671235e-09  3.0957130e-01  1.2411852e+00]
 [-3.9345893e-16  1.0000000e+00 -1.2491868e-08 -5.0084495e-08]
 [-3.0957130e-01  1.1878219e-08  9.5087630e-01  4.0124130e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]

Source view 9 -> Target view 0:
  delta_pitch: 0.00°
  delta_yaw:   17.08°
  delta_dist:  0.0000

Source view 9 -> Target view 5:
  delta_pitch: 0.00°
  delta_yaw:   7.60°
  delta_dist:  0.0000

Source view 9 -> Target view 10:
  delta_pitch: 0.00°
  delta_yaw:   -1.90°
  delta_dist:  0.0000

Source view 9 -> Target view 15:
  delta_pitch: 0.00°
  delta_yaw:   -11.40°
  delta_dist:  0.0000

--- Sample batch ---
jpg shape: torch.Size([4, 3, 256, 256])
hint shape: torch.Size([4, 3, 256, 256])
delta_pose shape: torch.Size([4, 3])
delta_pose sample values (radians): tensor([ 0.0000, -0.0332,  0.0000])
delta_pose sample values (degrees): pitch

In [6]:
# Training

from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader

transform = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    # T.Normalize([0.5], [0.5])
])

dataset = Portrait4dDataset(
    root_dir="datasets/portrait4d", transform=transform
)

dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_cb = ModelCheckpoint(
    dirpath="checkpoints/v2/",
    filename="toss-{step}",
    save_last=True,
    every_n_train_steps=500
)

trainer = pl.Trainer(
    max_steps=3000,
    accelerator="gpu",
    devices=1,
    callbacks=[checkpoint_cb],
    log_every_n_steps=10,
    gradient_clip_val=1.0,
    enable_checkpointing=True,
)

model.learning_rate = 1e-5
model.sd_locked = True
model.first_stage_key = "jpg"
model.control_key = "hint"
model.cond_stage_key = "txt"

import torch.utils.checkpoint as cp
cp.checkpoint = lambda func, *args, **kwargs: func(*args)

trainer.fit(model, dataloader)

# print("Initializing iterator...")
# data_iter = iter(dataloader)

# print("Fetching first batch...")
# batch = next(data_iter)
# print("Success! Batch keys:", batch.keys())

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\user\.conda\envs\toss\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
c:\Users\user\.conda\envs\toss\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
You are using a CUDA device ('NVIDIA GeForce RTX 4090

[OPT] LoRA param: base_model.model.middle_block.1.transformer_blocks.0.attn2.to_q.lora_A.default.weight, shape=torch.Size([16, 1280]), requires_grad=True
[OPT] LoRA param: base_model.model.middle_block.1.transformer_blocks.0.attn2.to_q.lora_B.default.weight, shape=torch.Size([1280, 16]), requires_grad=True
[OPT] LoRA param: base_model.model.middle_block.1.transformer_blocks.0.attn2.to_k.lora_A.default.weight, shape=torch.Size([16, 768]), requires_grad=True
[OPT] LoRA param: base_model.model.middle_block.1.transformer_blocks.0.attn2.to_k.lora_B.default.weight, shape=torch.Size([1280, 16]), requires_grad=True
[OPT] LoRA param: base_model.model.middle_block.1.transformer_blocks.0.attn2.to_v.lora_A.default.weight, shape=torch.Size([16, 768]), requires_grad=True
[OPT] LoRA param: base_model.model.middle_block.1.transformer_blocks.0.attn2.to_v.lora_B.default.weight, shape=torch.Size([1280, 16]), requires_grad=True
[OPT] LoRA param: base_model.model.output_blocks.3.1.transformer_blocks.0.attn

AttributeError: 'TossLoraModule' object has no attribute 'decode_latents'

In [7]:
import peft
print(peft.__version__)

0.17.1


In [1]:
# Load Model (TOSS original)

from PIL import Image
import numpy as np
import torchvision.transforms as T

# Load the last checkpoint
import torch
from cldm.toss import TOSS
from cldm.model import load_state_dict
from omegaconf import OmegaConf
from PIL import Image
from torchvision import transforms as T

from ldm.util import instantiate_from_config

config = OmegaConf.load("models/toss_vae.yaml")
model = instantiate_from_config(config.model)

state_dict = torch.load("ckpt/toss.ckpt", map_location="cpu", weights_only=False)["state_dict"]

# Remove the old pose_net so your new one can take its place
# keys_to_remove = [k for k in state_dict.keys() if "pose_net" in k]
# for k in keys_to_remove:
#     del state_dict[k]

model.load_state_dict(state_dict, strict=False)

# trained_ckpt = torch.load("checkpoints/v2/last-v1.ckpt", map_location="cpu", weights_only=False)

# This will inject your LoRA weights and your trained PoseNet
# m, u = model.load_state_dict(trained_ckpt["state_dict"], strict=False)

# print("Unexpected keys (should be empty):", u)

c:\Users\user\.conda\envs\toss\lib\site-packages\lightning_fabric\__init__.py:29: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)
c:\Users\user\.conda\envs\toss\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No module 'xformers'. Proceeding without it.
TossLoraModule: Running in eps-prediction mode


c:\Users\user\.conda\envs\toss\lib\site-packages\pytorch_lightning\utilities\distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v2.0.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(


DiffusionWrapper has 863.65 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
[INIT] Enabled requires_grad for 96 LoRA parameters
trainable params: 1,204,100 || all params: 864,838,340 || trainable%: 0.1392


_IncompatibleKeys(missing_keys=['model.diffusion_model.base_model.model.time_embed.0.weight', 'model.diffusion_model.base_model.model.time_embed.0.bias', 'model.diffusion_model.base_model.model.time_embed.2.weight', 'model.diffusion_model.base_model.model.time_embed.2.bias', 'model.diffusion_model.base_model.model.input_blocks.0.0.weight', 'model.diffusion_model.base_model.model.input_blocks.0.0.bias', 'model.diffusion_model.base_model.model.input_blocks.1.0.in_layers.0.weight', 'model.diffusion_model.base_model.model.input_blocks.1.0.in_layers.0.bias', 'model.diffusion_model.base_model.model.input_blocks.1.0.in_layers.2.weight', 'model.diffusion_model.base_model.model.input_blocks.1.0.in_layers.2.bias', 'model.diffusion_model.base_model.model.input_blocks.1.0.emb_layers.1.weight', 'model.diffusion_model.base_model.model.input_blocks.1.0.emb_layers.1.bias', 'model.diffusion_model.base_model.model.input_blocks.1.0.out_layers.0.weight', 'model.diffusion_model.base_model.model.input_block

In [1]:
# Load Model (Inference)

from PIL import Image
import numpy as np
import torchvision.transforms as T

# Load the last checkpoint
import torch
from cldm.toss import TOSS
from cldm.toss_lora import TossLoraModule
from cldm.model import load_state_dict
from omegaconf import OmegaConf
from PIL import Image
from torchvision import transforms as T

from ldm.util import instantiate_from_config

config = OmegaConf.load("models/toss_vae.yaml")
model = instantiate_from_config(config.model)

print(isinstance(model, TossLoraModule))

state_dict = torch.load("ckpt/toss.ckpt", map_location="cpu", weights_only=False)["state_dict"]

# Remove the old pose_net so your new one can take its place
# keys_to_remove = [k for k in state_dict.keys() if "pose_net" in k]
# for k in keys_to_remove:
#     del state_dict[k]

model.load_state_dict(state_dict, strict=False)

for n, p in model.named_parameters():
   if "lora" in n.lower() or ".out." in n.lower():
       print(n, p.abs().mean().item())


trained_ckpt = torch.load("checkpoints/v2/last-v4.ckpt", map_location="cpu", weights_only=False)

lora_keys = [k for k in trained_ckpt["state_dict"].keys() if "lora" in k.lower()]
print(f"Found {len(lora_keys)} LoRA keys in checkpoint:")
for k in lora_keys:
    print(f"  {k}: {trained_ckpt['state_dict'][k].abs().mean().item():.6f}")

# This will inject your LoRA weights and your trained PoseNet
m, u = model.load_state_dict(trained_ckpt["state_dict"], strict=False)

print()

for n, p in model.named_parameters():
   if "lora" in n.lower() or ".out." in n.lower():
       print(n, p.abs().mean().item())

print("Missing:", m)
print("Unexpected keys (should be empty):", u)

c:\Users\user\.conda\envs\toss\lib\site-packages\lightning_fabric\__init__.py:29: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)
c:\Users\user\.conda\envs\toss\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No module 'xformers'. Proceeding without it.


c:\Users\user\.conda\envs\toss\lib\site-packages\pytorch_lightning\utilities\distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v2.0.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(


TossLoraModule: Running in eps-prediction mode
DiffusionWrapper has 863.65 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
[INIT] Fixing zero-initialized lora_B: base_model.model.middle_block.1.transformer_blocks.0.attn2.to_q.lora_B.default.weight
[INIT] Fixing zero-initialized lora_B: base_model.model.middle_block.1.transformer_blocks.0.attn2.to_k.lora_B.default.weight
[INIT] Fixing zero-initialized lora_B: base_model.model.middle_block.1.transformer_blocks.0.attn2.to_v.lora_B.default.weight
[INIT] Fixing zero-initialized lora_B: base_model.model.output_blocks.3.1.transformer_blocks.0.attn2.to_q.lora_B.default.weight
[INIT] Fixing zero-initialized lora_B: base_model.model.output_blocks.3.1.transformer_blocks.0.attn2.to_k.lora_B.default.weight
[INIT] Fixing zero-initialized lora_B: base_model.model.output_blocks.3.1.transformer_blocks.0.attn2.to_v.lora_B.d

In [2]:
trainable = [n for n, p in model.named_parameters() if p.requires_grad]
print("Trainable parameters:", len(trainable))
for n in trainable:
    print(n)

Trainable parameters: 16
model.diffusion_model.base_model.model.middle_block.1.transformer_blocks.0.attn2.to_q.lora_A.default.weight
model.diffusion_model.base_model.model.middle_block.1.transformer_blocks.0.attn2.to_q.lora_B.default.weight
model.diffusion_model.base_model.model.middle_block.1.transformer_blocks.0.attn2.to_k.lora_A.default.weight
model.diffusion_model.base_model.model.middle_block.1.transformer_blocks.0.attn2.to_k.lora_B.default.weight
model.diffusion_model.base_model.model.middle_block.1.transformer_blocks.0.attn2.to_v.lora_A.default.weight
model.diffusion_model.base_model.model.middle_block.1.transformer_blocks.0.attn2.to_v.lora_B.default.weight
model.diffusion_model.base_model.model.output_blocks.3.1.transformer_blocks.0.attn2.to_q.lora_A.default.weight
model.diffusion_model.base_model.model.output_blocks.3.1.transformer_blocks.0.attn2.to_q.lora_B.default.weight
model.diffusion_model.base_model.model.output_blocks.3.1.transformer_blocks.0.attn2.to_k.lora_A.default.w

In [ ]:
# Inference v1 (Manually typed inference)
model.cuda()
model.eval()

# source image
img_name = "00010.jpg"
src_img = Image.open(f"examples/{img_name}").convert("RGB")

transform = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3)  # → [-1,1]
])

src_img = transform(src_img)
src_img = src_img.unsqueeze(0).cuda()
print(src_img.shape)

with torch.no_grad():
    z = model.encode_first_stage(src_img)
    z = model.get_first_stage_encoding(z)
print(z.shape)

# with torch.no_grad():
#     hint_post = model.encode_first_stage(src_img)
#     hint_latent = hint_post.mode() if hasattr(hint_post, "mode") else hint_post



# pose (example)
# delta_pose = torch.eye(4)[:3]   # (3,4)
# delta_pose = delta_pose.unsqueeze(0).cuda()
# delta_pose = torch.tensor([[0.0, 0.0, 0.0]], device="cuda")  # [B=1, 3]
# delta_pose = create_rotation_matrix(yaw_deg=30).cuda()

print("pose_net in_features:", model.model.diffusion_model.pose_net[0].in_features)

yaw_deg = 180.0
yaw = yaw_deg * torch.pi / 180
delta_pose = torch.tensor([[yaw, 45.0, 180.0]])
# delta_pose = torch.zeros(1, 16).cuda()
# delta_pose = torch.randn(1, 16).cuda() * 0.5
# delta_pose = torch.tensor([[9.50876296e-01, 3.86712351e-09,  3.09571296e-01,  1.24118519e+00,
#   -3.93458927e-16,  1.00000000e+00, -1.24918680e-08, -5.00844948e-08,
#   -3.09571296e-01,  1.18782193e-08,  9.50876296e-01,  4.01241302e+00,
#    0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]], device="cuda")
print("DELTA_POSE:", delta_pose.shape)

# delta_pose_cfg = delta_pose.repeat(2, 1) # Shape becomes [2, 16]


# empty text
txt = [""]

batch = {
        "jpg": src_img,
        "hint": src_img,
        # 'mask': mask,
        "delta_pose": delta_pose, # pose from original angle
        # "subject_id": os.path.basename(sub_path),
        "txt": txt
    }

print(src_img.shape)
print(delta_pose.shape)

x_T = torch.randn_like(z)

# cond = {
#     "in_concat": [z],
#     "c_crossattn": [model.get_learned_conditioning(txt)],
#     "c_concat": [src_img],
#     "delta_pose": delta_pose,
# }

# uc_cond = {
#     "in_concat": [z * 0],
#     "c_crossattn": [model.get_unconditional_conditioning(1)],
#     "c_concat": [src_img],
#     "delta_pose": delta_pose,
# }

x, cond = model.get_input(batch, model.first_stage_key)
# cond["delta_pose"] = torch.zeros_like(cond["delta_pose"])
print("COND", cond)

from ldm.models.diffusion.ddim import DDIMSampler
from torchvision.utils import save_image

sampler = DDIMSampler(model)

# with torch.no_grad():
#     samples, _ = sampler.sample(
#         S=50,                      # inference steps
#         batch_size=1,
#         shape=(4, 64, 64),         # latent shape (예시)
#         conditioning=cond,         # conditioning (same as training)
#         unconditional_guidance_scale=7.5,
#         unconditional_conditioning=uc_cond,
#         eta=0.0
#     )

# txt2img
# with torch.no_grad():
#     samples, _ = sampler.sample(
#         S=20,                      # inference steps
#         batch_size=1,
#         shape=(z.shape[1], z.shape[2], z.shape[3]),         # latent shape
#         conditioning=cond,         # conditioning (same as training)
#         unconditional_guidance_scale=1,
#         # unconditional_conditioning=uc_cond,
#         # eta=0.0
#     )

print("loaded?", any((p.abs().mean().item() > 0) for p in model.parameters()))
print("z:", z.shape, z.abs().mean().item(), z.std().item())
print("cond keys:", cond.keys(), {k: type(v) for k,v in cond.items()})

# img2img
sampler.make_schedule(
    ddim_num_steps=50,
    ddim_eta=0.0,
    verbose=False
)

print(sampler.ddim_timesteps)

strength = 0.4
t_enc = int(strength * len(sampler.ddim_timesteps))
z_enc = sampler.stochastic_encode(z, torch.tensor([t_enc]).cuda())

print("x_dec batch:", z_enc.shape[0])
print("cond delta_pose batch:", cond["delta_pose"].shape[0])
print("cond crossattn batch:", cond["c_crossattn"][0].shape[0])
print("cond in_concat batch:", cond["in_concat"][0].shape[0])


# Decode
samples = sampler.decode(
    z_enc,
    cond,
    t_enc,
    unconditional_guidance_scale=3.0,
    unconditional_conditioning=None
)

# Decode
with torch.no_grad():
    out = model.decode_first_stage(samples)
out = torch.clamp((out + 1) / 2, 0, 1)
print(out.shape)
save_image(out, "test_output.png")


# LOSS
    # Perceptual Loss, Contrastive Loss






torch.Size([1, 3, 256, 256])
torch.Size([1, 4, 32, 32])
pose_net in_features: 51
DELTA_POSE: torch.Size([1, 3])
torch.Size([1, 3, 256, 256])
torch.Size([1, 3])
get_input raw control: torch.Size([1, 3, 256, 256])
COND {'c_crossattn': [tensor([[[-0.3884,  0.0229, -0.0522,  ..., -0.4899, -0.3066,  0.0675],
         [-0.3711, -1.4497, -0.3401,  ...,  0.9489,  0.1867, -1.1034],
         [-0.5107, -1.4629, -0.2926,  ...,  1.0419,  0.0701, -1.0284],
         ...,
         [ 0.5006, -0.9552, -0.6610,  ...,  1.6013, -1.0622, -0.2191],
         [ 0.4988, -0.9451, -0.6656,  ...,  1.6467, -1.0858, -0.2088],
         [ 0.4924, -0.8124, -0.4912,  ...,  1.6108, -1.0174, -0.2484]]],
       device='cuda:0')], 'c_concat': [tensor([[[[ 0.6863,  0.7176,  0.7098,  ...,  0.7098,  0.7098,  0.6863],
          [ 0.7098,  0.7176,  0.7098,  ...,  0.7098,  0.7098,  0.7098],
          [ 0.7098,  0.7176,  0.7098,  ...,  0.7098,  0.7098,  0.7176],
          ...,
          [ 0.7255,  0.7255,  0.7098,  ...,  0.3490,  

Decoding image: 100%|██████████| 20/20 [00:00<00:00, 37.45it/s]

torch.Size([1, 3, 256, 256])


In [ ]:
# Inference v2 (Inference code used in Gradio)
# TOSS app.py to ipynb
model.cuda()
model.eval()

# source image
img_name = "00010.jpg"
src_img = Image.open(f"examples/{img_name}").convert("RGB")

transform = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    # T.Normalize([0.5]*3, [0.5]*3)  # → [-1,1]
])

src_img = transform(src_img)
src_img = src_img.unsqueeze(0).cuda()
print(src_img.shape)

with torch.no_grad():
    z = model.encode_first_stage(src_img)
    z = model.get_first_stage_encoding(z)
print(z.shape)

print("pose_net in_features:", model.model.diffusion_model.pose_net[0].in_features)

# empty text
txt = [""]

# batch = {
#         "jpg": src_img,
#         "hint": src_img,
#         # 'mask': mask,
#         "delta_pose": delta_pose, # pose from original angle
#         # "subject_id": os.path.basename(sub_path),
#         "txt": txt
#     }

n_samples = 1
prompt_scale = 0.0
img_scale = 1.0
ddim_steps = 50
ddim_eta = 0.0
h = 256
w = 256
# yaw_deg = 0
# yaw = yaw_deg * torch.pi / 180

# delta_pose = torch.tensor([[9.50876296e-01, 3.86712351e-09,  3.09571296e-01,  1.24118519e+00,
#   -3.93458927e-16,  1.00000000e+00, -1.24918680e-08, -5.00844948e-08,
#   -3.09571296e-01,  1.18782193e-08,  9.50876296e-01,  4.01241302e+00,
#    0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]], device="cuda")

import math
def get_T_from_relative(x, y, z, pose_enc="freq")->torch.Tensor:
    """
    Args:
        x: relative polar degree
        y: relative azimuth degree
        z: relative distance
        
    example:
        (0., -90., 0.): left view
        (0., 90., 0.): right view
        (0., 180., 0.): back view
        (-90., 0., 0.): top view
        (90., 0., 0.): bottom view
    """
    print("POSE_ENC:", pose_enc)
    if pose_enc in ["freq","identity", "vae"]:
        d_T = torch.tensor([math.radians(x), math.radians(y), z])
    elif pose_enc == "zero":
        d_T = torch.tensor([math.radians(x), math.sin(
                math.radians(y)), math.cos(math.radians(y)), z])
    else:
        raise NotImplementedError
    return d_T


delta_pose = torch.zeros((1, 3), device="cuda")
yaw_deg = -20.0
yaw = yaw_deg * torch.pi / 180
delta_pose = torch.tensor([[0, 0, 0]], device="cuda")

# print("DELTA_POSE:", delta_pose.shape)

# delta_pose = get_T_from_relative(0, 0, 0, pose_enc=model.model.diffusion_model.pose_enc)

# delta_pose_cfg = delta_pose.repeat(2, 1) # Shape becomes [2, 16]


# hint
c_cat = src_img
# text
uc_cross = model.get_unconditional_conditioning(n_samples)
c = model.get_learned_conditioning(txt)
# camera pose
# delta_pose = T[None, :].repeat(n_samples, 1).to(c.device)
# concat for concat pipline
in_concat = model.encode_first_stage(((src_img*2-1).to(c.device))).mode().detach()


cond = {}
cond['delta_pose'] = delta_pose
cond['c_crossattn'] = [c]
cond['c_concat'] = [c_cat]
cond['in_concat'] = [in_concat]

# uc2 for prompt
uc2 = {}
uc2['delta_pose'] = delta_pose
uc2['c_crossattn'] = [uc_cross]
uc2['c_concat'] = [c_cat]
uc2['in_concat'] = [in_concat]
            
# uc for image
uc = {}
uc['delta_pose'] = delta_pose
uc['c_crossattn'] = [uc_cross]
uc['c_concat'] = [c_cat]
uc['in_concat'] = [in_concat*0] 

from ldm.models.diffusion.ddim import DDIMSampler
from torchvision.utils import save_image

sampler = DDIMSampler(model)

shape = [4, h // 8, w // 8]
x_T = torch.randn(in_concat.shape, device=c.device)
samples_ddim, _ = sampler.sample(S=ddim_steps,
                                conditioning=cond,
                                batch_size=n_samples,
                                shape=shape,
                                verbose=False,
                                unconditional_guidance_scale=img_scale,
                                unconditional_conditioning=uc,
                                unconditional_guidance_scale2=prompt_scale,
                                unconditional_conditioning2=uc2,
                                eta=ddim_eta,
                                x_T=x_T)
print(samples_ddim.shape)
x_samples_ddim = model.decode_first_stage(samples_ddim)

out = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0).cpu()

print(out.shape)
save_image(out, "test_output.png")

torch.Size([1, 3, 256, 256])
torch.Size([1, 4, 32, 32])
pose_net in_features: 51
Data shape for DDIM sampling is (1, 4, 32, 32), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:01<00:00, 36.57it/s]

torch.Size([1, 4, 32, 32])
torch.Size([1, 3, 256, 256])
